In [4]:
class world:
    def __init__(self, n):
        self.people = []
        self.farmer_pop = int(round(n/3))
        self.bandit_pop = int(round(n/3))
        self.soiler_pop = int(round(n/3))

        for i in range(n):
            if i % 3 == 0:
                self.people.append(Person('farmer'))
            if i % 3 == 1:
                self.people.append(Person('bandit'))
            if i % 3 == 2:
                self.people.append(Person('soilder'))

    
    def update():
        for person in self.people:
            if person.role == 'farmer':
                person.apply_income_farmer()
                person.loyalty = 1 / self.farmer_pop
                
        for person in self.people:
            if person.role == 'bandit':
                person.bandit_action()
                person.loyalty = 1 / self.bandit_pop
                
        for person in self.people:
            if person.role == 'soilder':
                person.soilder_action()
                person.loyalty = 1 / self.soilder_pop
                
        for person in self.people:
            if person.role == 'farmer':
                person.apply_tax(0.2)
                
        for person in self.people:
            person.update_psychology()

        for person in self.people:
            person.decide_switch(group_loyalty_for_current_role = person.loyalty, utilities, switch_urge_threshold, utility_margin)

        self.farmer_pop = 0
        self.bandit_pop = 0
        self.soiler_pop = 0
        for person in self.people:
            person.finalize_role_switch():
            person.role = person.next_role

            if person.role == 'farmer':
                self.farmer_pop += 1
            if person.role == 'bandit':
                self.bandit_pop += 1
            if person.role == 'soilder':
                self.soilder_pop += 1

SyntaxError: positional argument follows keyword argument (2659098829.py, line 41)